# !pip install ultralytics

In [ ]:
!pip install pytorch

In [6]:
!pip install ultralytics easyocr opencv-python pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 17.7 MB/s eta 0:00:00


In [7]:
!pip install ultralytics paddleocr paddlepaddle opencv-python pandas pillow openpyxl


In [ ]:
# charger le modele fineturné

# **Code pour extraire et afficher des plaques d'immatriculation**

In [2]:
from ultralytics import YOLO
import os
import cv2
import easyocr
import pandas as pd
from PIL import Image
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage

# Charger le modèle YOLO
model = YOLO("My_best.pt")
# perform object detection on an image
results = model("dakar_auto_car_sale_79004.jpg", save =  True)
results[0].show()


image 1/1 /Users/modunjie/Desktop/DIT DATA SCIENCE/DEEP LEARNING/Detection dobjet/Plaques detections/dakar_auto_car_sale_79004.jpg: 576x640 2 License_Plates, 75.4ms
Speed: 4.7ms preprocess, 75.4ms inference, 10.8ms postprocess per image at shape (1, 3, 576, 640)
Results saved to runs/detect/predict9


In [4]:
from ultralytics import YOLO
import os
import cv2
import easyocr
import pandas as pd
from PIL import Image
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage

# Charger le modèle YOLO
model = YOLO("My_best.pt")

# Initialiser le lecteur EasyOCR
reader = easyocr.Reader(['en'], gpu=True)

# Dossier contenant les images
image_folder = "trafic_imag"

# Dossier pour sauvegarder les images recadrées
cropped_folder = "cropped_plates"
os.makedirs(cropped_folder, exist_ok=True)

# Lister les fichiers d'images dans le dossier
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Vérifier si des images sont disponibles
if not image_files:
    print("Aucune image trouvée dans le dossier.")
else:
    # Créer un DataFrame pour les données
    plate_data = []

    # Boucle sur toutes les images détectées
    for idx, image_path in enumerate(image_files):
        print(f"Traitement de l'image {idx + 1}/{len(image_files)} : {image_path}")

        # Charger l'image avec OpenCV
        image = cv2.imread(image_path)

        # Effectuer la détection avec YOLO
        results = model(image_path, save=True)

        # Parcourir les boîtes détectées
        for box_idx, box in enumerate(results[0].boxes):
            # Coordonnées de la boîte englobante
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            # Recadrer la zone détectée
            cropped_plate = image[y1:y2, x1:x2]

            # Sauvegarder l'image recadrée
            cropped_path = os.path.join(cropped_folder, f"plate_{idx}_{box_idx}.jpg")
            cv2.imwrite(cropped_path, cropped_plate)

            # Utiliser EasyOCR pour lire le texte
            ocr_results = reader.readtext(cropped_plate)

            # Extraire et enregistrer le texte
            for (bbox, text, prob) in ocr_results:
                plate_data.append({
                    "Image Name": os.path.basename(image_path),
                    "Plate Image": cropped_path,
                    "Text": text.strip(),
                    "Confidence": prob
                })

    # Convertir les données en DataFrame
    df = pd.DataFrame(plate_data)

    # Créer un fichier Excel avec les images des plaques
    excel_path = "detected_plates_with_images.xlsx"
    wb = Workbook()
    ws = wb.active
    ws.title = "Detected Plates"

    # Ajouter les en-têtes
    ws.append(["Image Name", "Plate Image", "Text", "Confidence"])

    # Ajuster la largeur des colonnes et la hauteur des lignes pour la lisibilité
    ws.column_dimensions["A"].width = 20  # Colonne Image Name
    ws.column_dimensions["B"].width = 30  # Colonne Plate Image
    ws.column_dimensions["C"].width = 50  # Colonne Text
    ws.column_dimensions["D"].width = 15  # Colonne Confidence
    for i in range(2, len(df) + 2):
        ws.row_dimensions[i].height = 50  # Hauteur des lignes

    # Ajouter les données avec les images
    for i, row in df.iterrows():
        ws.append([row["Image Name"], None, row["Text"], row["Confidence"]])

        # Insérer l'image dans la colonne "Plate Image"
        img = ExcelImage(row["Plate Image"])
        img.width, img.height = 120, 50  # Taille de la miniature
        ws.add_image(img, f"B{i + 2}")  # Colonne B, ligne correspondante

    # Sauvegarder le fichier Excel
    wb.save(excel_path)
    print(f"Fichier Excel généré avec succès : {excel_path}")


Traitement de l'image 1/5 : trafic_imag/medium_with_watermark_renault-duster-dakar-dakar-18580.jpeg

image 1/1 /Users/modunjie/Desktop/DIT DATA SCIENCE/DEEP LEARNING/Detection dobjet/Plaques detections/trafic_imag/medium_with_watermark_renault-duster-dakar-dakar-18580.jpeg: 416x640 1 License_Plate, 90.2ms
Speed: 12.8ms preprocess, 90.2ms inference, 10.7ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict10
Traitement de l'image 2/5 : trafic_imag/screenshot_27129_jpg.rf.c67a199f07c98e2e2183870698472a3b.jpg

image 1/1 /Users/modunjie/Desktop/DIT DATA SCIENCE/DEEP LEARNING/Detection dobjet/Plaques detections/trafic_imag/screenshot_27129_jpg.rf.c67a199f07c98e2e2183870698472a3b.jpg: 640x640 (no detections), 77.2ms
Speed: 1.3ms preprocess, 77.2ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict10
Traitement de l'image 3/5 : trafic_imag/screenshot_16709_jpg.rf.e5f7f17be1b8324a08bfd8575cc9efaf.jpg

image 1/

In [6]:
from ultralytics import YOLO
from paddleocr import PaddleOCR
import cv2
import os
import pandas as pd
from openpyxl import Workbook
from openpyxl.drawing.image import Image as ExcelImage

# Charger le modèle YOLO
model = YOLO("My_best.pt")

# Initialiser PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Ajouter d'autres langues si nécessaire

# Chemin de la vidéo
video_path = "a.mp4"  # Remplacez par le chemin de votre vidéo
output_folder = "detected_plates"
os.makedirs(output_folder, exist_ok=True)

# Charger la vidéo
cap = cv2.VideoCapture(video_path)

# Vérifier si la vidéo est chargée correctement
if not cap.isOpened():
    print(f"Erreur : Impossible de charger la vidéo {video_path}")
else:
    frame_count = 0
    plate_count = 0
    plate_data = []  # Stocker les informations des plaques pour Excel

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Effectuer la détection toutes les 5 frames pour réduire la charge
        if frame_count % 5 == 0:
            results = model(frame)

            # Parcourir les boîtes détectées
            for box_idx, box in enumerate(results[0].boxes):
                # Coordonnées de la boîte englobante
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                # Recadrer la zone détectée
                cropped_plate = frame[y1:y2, x1:x2]

                # Sauvegarder l'image recadrée
                plate_path = os.path.join(output_folder, f"plate_{frame_count}_{plate_count}.jpg")
                cv2.imwrite(plate_path, cropped_plate)
                plate_count += 1

                # Effectuer l'OCR sur la plaque détectée
                ocr_results = ocr.ocr(cropped_plate, cls=True)
                if ocr_results[0]:
                    for line in ocr_results[0]:
                        text, confidence = line[1][0], line[1][1]
                        plate_data.append({
                            "Frame": frame_count,
                            "Plate Image": plate_path,
                            "Text": text.strip(),
                            "Confidence": confidence
                        })

    # Libérer les ressources vidéo
    cap.release()

    # Sauvegarder les informations dans un fichier Excel
    excel_path = "detected_plates_with_paddleocr.xlsx"
    wb = Workbook()
    ws = wb.active
    ws.title = "Detected Plates"

    # Ajouter les en-têtes
    ws.append(["Frame", "Plate Image", "Text", "Confidence"])

    # Ajuster les dimensions des colonnes et des lignes
    ws.column_dimensions["A"].width = 15  # Colonne Frame
    ws.column_dimensions["B"].width = 30  # Colonne Plate Image
    ws.column_dimensions["C"].width = 50  # Colonne Text
    ws.column_dimensions["D"].width = 15  # Colonne Confidence
    for i in range(2, len(plate_data) + 2):
        ws.row_dimensions[i].height = 50  # Hauteur des lignes

    # Ajouter les données avec les miniatures
    for i, row in enumerate(plate_data, start=2):
        ws.append([row["Frame"], None, row["Text"], row["Confidence"]])

        # Ajouter la miniature dans la colonne "Plate Image"
        img = ExcelImage(row["Plate Image"])
        img.width, img.height = 120, 50  # Taille de la miniature
        ws.add_image(img, f"B{i}")

    # Sauvegarder le fichier Excel
    wb.save(excel_path)
    print(f"Fichier Excel généré avec succès : {excel_path}")


[2024/12/06 22:09:56] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/modunjie/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/modunjie/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_t